In [1]:
import numpy as np
import cv2 as cv
from scipy.spatial.transform import Rotation

In [2]:
def quat_to_R(q):
    x,y,z,w = q
    s = np.linalg.norm([w,x,y,z])
    w,x,y,z = w/s, x/s, y/s, z/s
    R = np.array([
        [1-2*(y*y+z*z),   2*(x*y - z*w),   2*(x*z + y*w)],
        [2*(x*y + z*w),   1-2*(x*x+z*z),   2*(y*z - x*w)],
        [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x*x+y*y)]
    ])
    return R



# SHOUTOUT https://github.com/RealManRobot/hand_eye_calibration/blob/main/compute_to_hand.py
def convert(x ,y ,z, rotation_matrix, translation_vector):
    obj_camera_coordinates = np.array([x, y, z])
    T_camera_to_base_effector = np.eye(4)
    T_camera_to_base_effector[:3, :3] = rotation_matrix
    T_camera_to_base_effector[:3, 3] = translation_vector.reshape(3)

    # Compute the pose of the object against the base
    obj_camera_coordinates_homo = np.append(obj_camera_coordinates, [1])  # Convert object coordinates to homogeneous coordinates
    obj_base_effector_coordinates_homo = T_camera_to_base_effector.dot(obj_camera_coordinates_homo)
    obj_base_coordinates = obj_base_effector_coordinates_homo[:3]  

    rot_matrix_homo = T_camera_to_base_effector[:3, :3]
    quaternion = Rotation.from_matrix(rot_matrix_homo).as_quat()

    return list(obj_base_coordinates), quaternion


In [3]:
# --- 4 snapshots (ABB robtarget-like) ---
# base -> gripper
abb_pos_mm = [  # x, y, z in mm
    [100,   0, 200],   # pose 0
    [120,  20, 220],   # pose 1
    [ 80, -30, 250],   # pose 2
    [150,  10, 180],   # pose 3
]

# quaternions (x, y, z, w)
# pose0: identity
# pose1: +10° about Z
# pose2: +10° about X
# pose3: +10° about Y
s = 0.0871557   # sin(10°/2)
c = 0.9961947   # cos(10°/2)
abb_quat_xyzw = [
    [0.0, 0.0, 0.0, 1.0],    # pose 0
    [0.0, 0.0, s,   c],      # pose 1  (Z)
    [s,   0.0, 0.0, c],      # pose 2  (X)
    [0.0, s,   0.0, c],      # pose 3  (Y)
]


# rvec_t2c: target(board) -> camera  (Rodrigues, radians)
rvec_t2c = [
    [0.0,     0.0,     0.26180],  # ~15° yaw
    [0.0,     0.0,     0.52360],  # ~30° yaw
    [0.17453, 0.0,     0.26180],  # ~10° pitch + 15° yaw
    [0.0,     0.17453, 0.34907],  # ~10° roll  + 20° yaw
]

# t_t2c: target(board) -> camera (meters)
t_t2c = [
    [-100,  20,  500.0],
    [-120.0,  10.0,  500.0],
    [-90.0, -20.0,  520.0],
    [-110.0,  30.0,  480.0],
]


In [4]:
""" R_b2g = quat_to_R(abb_quat_xyzw)
t_b2g = [np.array(tv, np.float64).reshape(3,1) for tv in abb_pos_mm]
#print(R_b2g)
#print(t_b2g)

#R_g2b = R_b2g.T
#t_g2b = - R_b2g.T @ t_b2g
#print(R_g2b, t_g2b)


R_g2b, t_g2b = [], []
for pos, quat in zip(abb_pos_mm, abb_quat_xyzw):
    R_b2g = quat_to_R(quat)                                 # (3,3)
    t_b2g = np.asarray(pos, np.float64).reshape(3,1)  #  (3,1)
    # invert to gripper->base
    R_gb = R_b2g.T
    t_gb = - R_gb @ t_b2g
    R_g2b.append(R_gb)
    t_g2b.append(t_gb) """

' R_b2g = quat_to_R(abb_quat_xyzw)\nt_b2g = [np.array(tv, np.float64).reshape(3,1) for tv in abb_pos_mm]\n#print(R_b2g)\n#print(t_b2g)\n\n#R_g2b = R_b2g.T\n#t_g2b = - R_b2g.T @ t_b2g\n#print(R_g2b, t_g2b)\n\n\nR_g2b, t_g2b = [], []\nfor pos, quat in zip(abb_pos_mm, abb_quat_xyzw):\n    R_b2g = quat_to_R(quat)                                 # (3,3)\n    t_b2g = np.asarray(pos, np.float64).reshape(3,1)  #  (3,1)\n    # invert to gripper->base\n    R_gb = R_b2g.T\n    t_gb = - R_gb @ t_b2g\n    R_g2b.append(R_gb)\n    t_g2b.append(t_gb) '

In [5]:
#R_b2g = quat_to_R(abb_quat_xyzw)
#t_b2g = [np.array(tv, np.float64).reshape(3,1) for tv in abb_pos_mm]
#print(R_b2g)
#print(t_b2g)

#R_g2b = R_b2g.T
#t_g2b = - R_b2g.T @ t_b2g
#print(R_g2b, t_g2b)



def camera2robot_calib(position_robot, quat_robot, rotation_vec_t2c, translation_t2c):
    R_g2b, t_g2b = [], []
    for pos, quat in zip(position_robot, quat_robot): #position robot list of XYZ for each position of calibration and quat is its quaternion
        R_b2g = quat_to_R(quat)                                 # (3,3)
        t_b2g = np.asarray(pos, np.float64).reshape(3,1)  #  (3,1)
        # invert to gripper->base
        R_gb = R_b2g.T
        t_gb = - R_gb @ t_b2g
        R_g2b.append(R_gb)
        t_g2b.append(t_gb)

    R_t2c = [cv.Rodrigues(np.array(rv, np.float64))[0] for rv in rotation_vec_t2c]
    t_t2c = [np.array(tv, np.float64).reshape(3,1)     for tv in translation_t2c]

    R_cam2gripper, t_cam2gripper = cv.calibrateHandEye(
        R_gripper2base=R_g2b, t_gripper2base=t_g2b,
        R_target2cam=R_t2c,   t_target2cam=t_t2c,
        method=cv.CALIB_HAND_EYE_DANIILIDIS
    )
    return R_cam2gripper, t_cam2gripper

# 旋转矩阵
#rotation = Rotation.from_matrix(R_cam2gripper)
# 将旋转矩阵转换为四元数
#print("Rotation matrix, R_cam2gripper:\n", R_cam2gripper)
#print("Traslation vector, t_cam2gripper:\n", t_cam2gripper)
#print("Quaternion based on rotation matrix from cam2gripper: \n",quaternion)

R_cam2gripper, t_cam2gripper = camera2robot_calib(abb_pos_mm, abb_quat_xyzw, rvec_t2c, t_t2c)
print(R_cam2gripper, t_cam2gripper)

[[ 0.89841564 -0.06666992 -0.43405582]
 [ 0.25785393  0.88016718  0.39851862]
 [ 0.35547249 -0.46995836  0.80794706]] [[ 374.39972847]
 [ -44.7444923 ]
 [-505.59544599]]


In [6]:
convert(200, -100 , 115 , R_cam2gripper, t_cam2gripper)

([510.8334285533251, -35.36078412661436, -294.59120076445066],
 array([-0.22929315, -0.20844932,  0.08567999,  0.94690679]))

In [7]:
""" #
import numpy as np
import cv2 as cv
from scipy.spatial.transform import Rotation

def quat_to_R(q):
    x,y,z,w = q
    # normalize
    s = np.linalg.norm([w,x,y,z])
    w,x,y,z = w/s, x/s, y/s, z/s
    R = np.array([
        [1-2*(y*y+z*z),   2*(x*y - z*w),   2*(x*z + y*w)],
        [2*(x*y + z*w),   1-2*(x*x+z*z),   2*(y*z - x*w)],
        [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x*x+y*y)]
    ])
    return R


R_b2g = quat_to_R(abb_quat_xyzw)
t_b2g = [np.array(tv, np.float64).reshape(3,1) for tv in abb_pos_mm]
#print(R_b2g)
#print(t_b2g)

R_g2b = R_b2g.T
t_g2b = - R_b2g.T @ t_b2g
#print(R_g2b, t_g2b)



R_t2c = [cv.Rodrigues(np.array(rv, np.float64))[0] for rv in rvec_t2c]
t_t2c = [np.array(tv, np.float64).reshape(3,1)     for tv in t_t2c]

R_cam2gripper, t_cam2gripper = cv.calibrateHandEye(
    R_gripper2base=R_g2b, t_gripper2base=t_g2b,
    R_target2cam=R_t2c,   t_target2cam=t_t2c,
    method=cv.CALIB_HAND_EYE_DANIILIDIS
)

rotation = Rotation.from_matrix(R_cam2gripper)
quaternion = rotation.as_quat() # 

print("Rotation matrix, R_cam2gripper:\n", R_cam2gripper)
print("Traslation vector, t_cam2gripper:\n", t_cam2gripper)



 """

' #\nimport numpy as np\nimport cv2 as cv\nfrom scipy.spatial.transform import Rotation\n\ndef quat_to_R(q):\n    x,y,z,w = q\n    # normalize\n    s = np.linalg.norm([w,x,y,z])\n    w,x,y,z = w/s, x/s, y/s, z/s\n    R = np.array([\n        [1-2*(y*y+z*z),   2*(x*y - z*w),   2*(x*z + y*w)],\n        [2*(x*y + z*w),   1-2*(x*x+z*z),   2*(y*z - x*w)],\n        [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x*x+y*y)]\n    ])\n    return R\n\n\nR_b2g = quat_to_R(abb_quat_xyzw)\nt_b2g = [np.array(tv, np.float64).reshape(3,1) for tv in abb_pos_mm]\n#print(R_b2g)\n#print(t_b2g)\n\nR_g2b = R_b2g.T\nt_g2b = - R_b2g.T @ t_b2g\n#print(R_g2b, t_g2b)\n\n\n\nR_t2c = [cv.Rodrigues(np.array(rv, np.float64))[0] for rv in rvec_t2c]\nt_t2c = [np.array(tv, np.float64).reshape(3,1)     for tv in t_t2c]\n\nR_cam2gripper, t_cam2gripper = cv.calibrateHandEye(\n    R_gripper2base=R_g2b, t_gripper2base=t_g2b,\n    R_target2cam=R_t2c,   t_target2cam=t_t2c,\n    method=cv.CALIB_HAND_EYE_DANIILIDIS\n)\n\nrotation = Rotat